In [62]:
from sklearn.linear_model import LogisticRegression as lr
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
import math
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import seaborn as sns
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)

import networkx as nx
from networkx.algorithms.tree.branchings import maximum_branching
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

In [63]:
full_dataset_confounders_2011 = pd.read_csv('processed_data/full_dataset_confounders_2011.csv')
full_dataset_confounders_2012 = pd.read_csv('processed_data/full_dataset_confounders_2012.csv')
full_dataset_confounders_2013 = pd.read_csv('processed_data/full_dataset_confounders_2013.csv')

def convert_absolutes_to_percentages(dataset):
    df = dataset.copy()
    df['Aged 16-64 Percentage'] = df["Aged 16-64"] / (df["Aged 16-64"] + df["Aged 0-15"] + df["Aged 65+"]) * 100
    df['Aged 0-15 Percentage'] = df["Aged 0-15"] / (df["Aged 16-64"] + df["Aged 0-15"] + df["Aged 65+"]) * 100
    df['Aged 65+ Percentage'] = df["Aged 65+"] / (df["Aged 16-64"] + df["Aged 0-15"] + df["Aged 65+"]) * 100
    df["Percentage of Full-time employees"] = df["Number of Full-time employees"] / (df["Number of Part-time employees"] + df["Number of Full-time employees"]) * 100
    df["Percentage of Part-time employees"] = df["Number of Part-time employees"] / (df["Number of Part-time employees"] + df["Number of Full-time employees"]) * 100
    df.drop(['Aged 0-15', 'Aged 16-64', 'Aged 65+', 'Number of Full-time employees', 'Number of Part-time employees'], axis=1, inplace=True)
    df.fillna(0, inplace=True)
    return df
 
full_dataset_confounders_2011 = convert_absolutes_to_percentages(full_dataset_confounders_2011)
full_dataset_confounders_2012 = convert_absolutes_to_percentages(full_dataset_confounders_2012)
full_dataset_confounders_2013 = convert_absolutes_to_percentages(full_dataset_confounders_2013)

In [64]:
full_dataset_confounders_2011

,osward,QUANTITY,Area that is greenspace,Percentage homes with deficiency in access to nature,DWP Working-age client group (rates),Employment and support allowance claimants,Housing Benefit rates,Income Support Claimants,Incapacity Benefit Claimants,JSA Claimant Rate,...,outdoor_count,pub_count,skate_count,theatre_count,total_count,Aged 16-64 Percentage,Aged 0-15 Percentage,Aged 65+ Percentage,Percentage of Full-time employees,Percentage of Part-time employees
0,E05000026,136.609508,19.60720,2.164412,17.1,175.0,15.195767,525.0,305.0,8.525224,...,0.0,6.0,0.0,1.0,13.0,70.489156,24.526996,4.983849,65.822785,34.177215
1,E05000027,181.278351,22.41290,71.727362,22.7,150.0,19.040541,595.0,395.0,8.622620,...,0.0,1.0,0.0,0.0,1.0,63.276620,26.381189,10.342191,50.000000,50.000000
2,E05000028,310.103636,3.03888,17.166271,20.1,175.0,16.910180,590.0,440.0,9.123722,...,0.0,0.0,0.0,0.0,1.0,65.226763,25.383687,9.389550,45.454545,54.545455
3,E05000029,254.118297,56.40730,63.592351,21.1,115.0,14.816327,435.0,330.0,7.688611,...,0.0,5.0,0.0,0.0,7.0,59.984065,24.519470,15.496464,52.941176,47.058824
4,E05000030,201.110643,51.11650,0.000000,16.8,95.0,11.861635,340.0,260.0,8.021933,...,0.0,2.0,0.0,0.0,2.0,64.223236,21.087659,14.689104,72.500000,27.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,E05011485,229.847122,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,8.0,1.0,1.0,11.0,0.000000,0.000000,0.000000,0.000000,0.000000
580,E05011486,266.757530,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,2.0,0.0,0.0,2.0,0.000000,0.000000,0.000000,0.000000,0.000000
581,E05011487,491.851936,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,3.0,1.0,0.0,4.0,0.000000,0.000000,0.000000,0.000000,0.000000
582,E05011488,252.618917,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,3.0,0.0,0.0,3.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [65]:
# Get same results as in results2

def get_treatment_level(input, bins):
    if input < bins[0]:
        return 0
    elif input < bins[1]:
        return 1
    elif input < bins[2]:
        return 2
    else:
        return 3

def logit(p):
    logit_value = math.log(p / (1-p))
    return logit_value

def plot_propensity_plots(predictions, predictions_logit, T):
    fig, ax = plt.subplots(1,2)
    fig.suptitle('Density distribution plots for propensity score and logit(propensity score).')
    sns.kdeplot(x = predictions[:,1], hue = T , ax = ax[0])
    ax[0].set_title('Propensity Score')
    sns.kdeplot(x = predictions_logit, hue = T , ax = ax[1])
    ax[1].axvline(-0.4, ls='--')
    ax[1].set_title('Logit of Propensity Score')
    plt.show()

def create_maximum_branching_graph(df_data):
    G = nx.Graph()
    G.add_nodes_from(df_data.reset_index()['osward'].tolist())
    df_data_no_index = df_data.reset_index()
    epsilon = 0.0001
    for index, row in df_data_no_index.iterrows():
        other_rows = df_data_no_index[df_data_no_index['treatment'] != row.treatment]
        for o_index, o_row in other_rows.reset_index().iterrows():
            if not G.has_edge(row.osward, o_row.osward):
                modified_distance = (abs(row.propensity_score_logit - o_row.propensity_score_logit) + epsilon) / abs(row.treatment - o_row.treatment)
                G.add_edge(row.osward, o_row.osward, weight=modified_distance)
    return maximum_branching(G)

def plot_correlation(input_data, treatment_column):
    correlation = input_data[[treatment_column, 'outcome']]
    correlation.plot.scatter(x=treatment_column, y='outcome')
    spear_corr = stats.spearmanr(list(correlation[treatment_column]), list(correlation['outcome']))
    print('Spearman correlation:', spear_corr.correlation, 'p-value:', spear_corr.pvalue)

def plot_distance_distribution(edmonds_applied, df_data):
    distances = []
    treatment_effect = []
    for (u,v) in edmonds_applied.edges():
        distances.append(abs(df_data['treatment'][u] - df_data['treatment'][v]))
        effect = (df_data['outcome'][u] - df_data['outcome'][v])/(df_data['treatment'][u] - df_data['treatment'][v])
        treatment_effect.append(effect)
    distribution_data = pd.DataFrame(list(zip(distances,treatment_effect)), columns = ['Distance', 'Effect'])
    sns.jointplot(distribution_data, x='Distance', y='Effect', kind='kde', fill=True)

def print_distance_confusion_matrix(edmonds_applied, df_data):
    low_dose_units = []
    high_dose_units = []

    for (u,v) in edmonds_applied.edges():
        u_treatment = df_data['treatment'][u]
        v_treatment = df_data['treatment'][v]
        if u_treatment > v_treatment:
            low_dose_units.append(v_treatment)
            high_dose_units.append(u_treatment)
            continue
        low_dose_units.append(u_treatment)
        high_dose_units.append(v_treatment)

    print(confusion_matrix(high_dose_units, low_dose_units))

def obtain_results(input_data, treatment_column, outcome_column, columns_to_drop, bins):
    df = input_data.copy()
    subset = df.drop(columns_to_drop, axis=1)
    df[treatment_column] = df[treatment_column].astype('int')
    subset[treatment_column] = subset[treatment_column].astype('int')
    treatment_levels = [get_treatment_level(x, bins) for x in list(df[treatment_column])]
    subset_treatment_levels = [get_treatment_level(x, bins) for x in list(subset[treatment_column])]
    df[treatment_column + '_bin'] = treatment_levels
    subset[treatment_column] = subset_treatment_levels

    T = subset[treatment_column]
    X = subset.loc[:,subset.columns != treatment_column]
    y = input_data[[outcome_column]]

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('logistic_classifier', lr())
    ])
    pipe.fit(X, T)

    predictions = pipe.predict_proba(X)
    predictions_binary = pipe.predict(X)
    predictions_logit = np.array([logit(xi) for xi in predictions[:,1]])
    df.loc[:,'propensity_score'] = predictions[:,1]
    df.loc[:,'propensity_score_logit'] = predictions_logit
    df.loc[:,'outcome'] = y[outcome_column]

    X.loc[:,'propensity_score'] = predictions[:,1]
    X.loc[:,'propensity_score_logit'] = predictions_logit
    X.loc[:,'outcome'] = y[outcome_column]
    X.loc[:,'treatment'] = df[treatment_column + '_bin']

    caliper = np.std(df.propensity_score) * 0.25
    # print('\nCaliper (radius) is: {:.4f}\n'.format(caliper))

    df_data = X
    knn = NearestNeighbors(n_neighbors=10 , p = 2, radius=caliper)
    knn.fit(df_data[['propensity_score_logit']].to_numpy())
    
    distances , indexes = knn.kneighbors(
        df_data[['propensity_score_logit']].to_numpy(), \
        n_neighbors=10)
    df_data['osward'] = range(1, len(df_data) + 1)
    df_data.osward = 'E' + df_data.osward.astype(str)
    df_data.set_index('osward', inplace=True)

    edmonds_applied = create_maximum_branching_graph(df_data)
    treatment_effect = []
    for (u,v) in edmonds_applied.edges():
        effect = (df_data['outcome'][u] - df_data['outcome'][v])/(df_data['treatment'][u] - df_data['treatment'][v])
        treatment_effect.append(effect)
    return treatment_effect


In [66]:
# Helper functions for generating data

def get_truncnorm_generated(dataset, size):
    a = dataset.min()
    b = dataset.max()
    mu = dataset.mean()
    sigma = dataset.std()
    return stats.truncnorm.rvs(a, b, loc=mu, scale=sigma, size=size)

def get_exponential_generated(dataset, size):
    mu = dataset.mean()
    sigma = dataset.std()
    return stats.expon.rvs(loc=mu, scale=sigma, size=size)

def get_exponential_treatment_generated(row, a = 200):
    scale = row.sum() / a
    return stats.expon.rvs(loc = 0, scale=scale, size=1)[0]

def get_truncated_outcome_generated(row, confounders, a = 0.000015, b = 100, ate = -1):
    alpha = a * confounders.sum()
    beta = b * confounders.sum()
    mu = confounders.mean()
    sigma = confounders.std()
    return max(stats.truncnorm.rvs(a = alpha, b = beta, loc = mu, scale = sigma, size=1)[0] + ate * row['treatment'], 0)


In [70]:
def calculate_averages(effects, amt):
    ate = 0
    minimum = 0
    percentile_25 = 0
    percentile_75 = 0
    maximum = 0
    for eff in effects:
        ate += np.mean(eff)
        minimum += np.min(eff)
        percentile_25 += np.percentile(eff,25)
        percentile_75 += np.percentile(eff,75)
        maximum += np.max(eff)
    print('ATE:', ate/amt, 'Min:', minimum/amt,'25th %:', percentile_25/amt, '75th %:', percentile_75/amt, 'Max:', maximum/amt)


def get_average_treatment_over_runs(dataset, ate= -1, runs=10):
    simulated_dfs = []

    for _ in range(runs):
        age1664_generated = np.array(get_truncnorm_generated(dataset['Aged 16-64 Percentage'], 625))
        age015_generated = np.array(get_truncnorm_generated(dataset['Aged 0-15 Percentage'], 625))
        age65plus_generated = np.array(get_truncnorm_generated(dataset['Aged 65+ Percentage'], 625))
        percentage_greenspace_generated = np.array(get_truncnorm_generated(dataset["Area that is greenspace"], 625))
        percentage_working_age_rates = np.array(get_truncnorm_generated(dataset["DWP Working-age client group (rates)"], 625))
        employment_support_claimants = np.array(get_truncnorm_generated(dataset["Employment and support allowance claimants"], 625))
        housing_benefit_rates = np.array(get_truncnorm_generated(dataset["Housing Benefit rates"], 625))
        income_support_claimants = np.array(get_truncnorm_generated(dataset["Income Support Claimants"], 625))
        incapacity_benefit_claimants = np.array(get_truncnorm_generated(dataset["Incapacity Benefit Claimants"], 625))
        jsa_claimant_rate = np.array(get_truncnorm_generated(dataset["JSA Claimant Rate"], 625))

        full_time_employees = np.array(get_exponential_generated(dataset["Percentage of Full-time employees"], 625))
        part_time_employees = np.array(get_exponential_generated(dataset["Percentage of Part-time employees"], 625))
        deficiency_access_nature = np.array(get_exponential_generated(dataset["Percentage homes with deficiency in access to nature"], 625))
        
        simulated_confounders = [age1664_generated, age015_generated, age65plus_generated, percentage_greenspace_generated, percentage_working_age_rates, employment_support_claimants, housing_benefit_rates, income_support_claimants, incapacity_benefit_claimants, jsa_claimant_rate, full_time_employees, part_time_employees, deficiency_access_nature]
        simulated_confounders_df = pd.DataFrame(simulated_confounders).transpose()
        confounders = ['Aged 16-64 Percentage', 'Aged 0-15 Percentage', 'Aged 65+ Percentage', "Area that is greenspace", "DWP Working-age client group (rates)", "Employment and support allowance claimants", "Housing Benefit rates", "Income Support Claimants",
            "Incapacity Benefit Claimants", "JSA Claimant Rate", "Percentage of Part-time employees", "Percentage of Full-time employees", "Percentage homes with deficiency in access to nature"]
        simulated_confounders_df.columns = confounders
        simulated_confounders_df['treatment'] = simulated_confounders_df.apply(lambda row: get_exponential_treatment_generated(row), axis=1)
        simulated_confounders_df['outcome'] = simulated_confounders_df.apply(lambda row: get_truncated_outcome_generated(row, row[confounders], ate=ate), axis=1)
        simulated_dfs.append(simulated_confounders_df)
    effects = []
    for simulated_df in simulated_dfs:
        effects.append(obtain_results(simulated_df, 'treatment', 'outcome', ['outcome'], bins=[4,7,12]))
    calculate_averages(effects, runs)


In [72]:
get_average_treatment_over_runs(full_dataset_confounders_2012, ate=-1, runs=10)